# Grouping

We saw in our cursory review of the descriptives that there is evidence the data is skewed. We also saw that there are some other interesting patterns to the data. We saw for example that the range of scores was quite large. However, we also saw that there appeared to a large amount of intra-statistic-consistency across each of the areas measured (Open, High, Low etc.). 

Ahead of further analysis we want to understnd these patterns better, such that we better frame further tests. It might be that there are companies we want to exclude, it may be that there are specific time period that impact the data. These findings might tell us we need to transform the data for tests we will later use. We may find that the year has an impact on the data also, which will further change how we want to analyse further.

Importantly, there are 500 companies in the data with 9 columns of values daily over a 5 year period. This is equivalent to approximately **5,850,000** individual values. This poses a significant computational challenge. We therefore need to find a way to minimize this number so that we can more effectively compute. We will therefore average high and low scores for each company and use only these means. 

We will then Find the first and fourth quartiles of this data. We can then compare these score and more importantly, test to see if either the fourth or the 1st quartile differ significntly enough from the second and third quartiles.

**We will do the following:**
- Find the top quartile of companies
- Find the bottom quartile of companies
- Compare first and fourth with the second and third
- Compute the interquartile range (IQR)
    - Exclude companies that can be classified as outliers using the IQR
- If there are no outliers, run comparisons 
    - Exclude companies that skew the data the most
- Re-run means and medians without outliers
- Compare descriptives for years
- Test skewness and goodness of fit

In [9]:
# Import modules and variables we need 
import numpy as np
import pandas as pd
yfin_csv = pd.read_csv(r'https://raw.githubusercontent.com/alexcrockett/Jupyter-Playground/personal/yfin_dataset/02-data/stock_details_5_years.csv')
import yfin_descriptives_py # import our previous work
from yfin_descriptives_py import x_1, x_2, x_3, x_4, x_5, x_6, x_7 # import arrays
from yfin_descriptives_py import di_mean1, di_median1, di_max_min_range, di_var1, di_std1 # import disctionaries

In [10]:
# Finding the fourth quartile of data

## Compute the average between high and low for each company:
yfin_csv['Average'] = (yfin_csv['High'] + yfin_csv['Low']) / 2 # Here we are defining a new column called Average

## Group by company and compute the mean of the 'Average' column
company_averages = yfin_csv.groupby('Company')['Average'].mean() # Now we create a new variable to group companies with their means and compute means for these groups

## Find the quartiles
first_quartile = company_averages.quantile(0.25) # Define the 1st quartile
fourth_quartile = company_averages.quantile(0.75) #Define the 4th quartile

## Segment the quartiles
companies_in_first_quartile = company_averages[company_averages <= first_quartile] # Identify those in the 1st quartile
companies_in_fourth_quartile = company_averages[company_averages >= fourth_quartile] # Identify those in the 4th quartile

## Print the first quartile
print("Companies in the first quartile:")
print(companies_in_first_quartile)
print("-------------------")

print("Companies in the fourth quartile:")
print(companies_in_fourth_quartile)
print("-------------------")

# Sort the data

## Sort companies in the first quartile in ascending order to find the lowest
lowest_in_first_quartile = companies_in_first_quartile.sort_values(ascending=True)
highest_in_first_quartile = companies_in_first_quartile.sort_values(ascending=False)

## Sort companies in the fourth quartile in descending order to find the highest
lowest_in_fourth_quartile = companies_in_fourth_quartile.sort_values(ascending=True)
highest_in_fourth_quartile = companies_in_fourth_quartile.sort_values(ascending=False)

## Print the lowest company in the first quartile
print("Lowest company in the first quartile:")
print(lowest_in_first_quartile.head(1))  # .head(1) gets the top company after sorting in ascending order
print("-------------------")

## Print the highest company in the first quartile
print("Highest in the first quartile")
print(highest_in_first_quartile.head(1))
print("-------------------")

## Print the lowest company in the fourth quartile
print("Lowest company in the fourth quartile:")
print(lowest_in_fourth_quartile.head(1))  # .head(1) gets the top company
print("-------------------")


# Print the highest company in the fourth quartile
print("Highest company in the fourth quartile:")
print(highest_in_fourth_quartile.head(1))  # .head(1) gets the top company
print("-------------------")


Companies in the first quartile:
Company
ABEV     2.947499
BAC     30.790636
BBD      4.368681
BBVA     4.879372
BCE     40.470195
          ...    
WFC     39.525165
WIT      5.356330
WMB     23.720061
WPM     36.569403
WY      28.026198
Name: Average, Length: 123, dtype: float64
-------------------
Companies in the fourth quartile:
Company
ACN     249.476417
ADBE    417.060667
ADP     184.081445
ADSK    215.628537
AMGN    212.491112
           ...    
VRTX    247.068470
WDAY    202.697743
WST     270.453652
WTW     204.263422
ZM      173.939238
Name: Average, Length: 123, dtype: float64
-------------------
Lowest company in the first quartile:
Company
LYG    2.102133
Name: Average, dtype: float64
-------------------
Highest in the first quartile
Company
BK    42.263321
Name: Average, dtype: float64
-------------------
Lowest company in the fourth quartile:
Company
VMC    155.851637
Name: Average, dtype: float64
-------------------
Highest company in the fourth quartile:
Company
NVR  

# Notes on medians
In the above code we created a new column and created an average, for each row, based on the highest and lowest scores. We then grouped scores by company and then took the mean for each group. This gave us a smaller set of data to work with. We then computed the 1st and 4th quartiles. 

Even though there may be some loss of fidelity in this view, it gives us a view into the companies that may be having an effect on the data.We can use this to make decisions about the original dat set.

We can see that the company with the lowest mean is LYG with 2.10 USD and the highest mean is NVR with 4369.58 USD.

In the notebook 'fin_descriptives.ipynb' we saw that the average low score was 138.28 USD and the average high score was 141.85 (variables mean_low and mean_high respectively). In other words our lowest score is  136.18 USD from the mean and the highest is 4,227.73 from mean_high. 

In fact looking at the highest value in the 1st quartile and the lowest value in the fourth quartile gives a little more color to the data. The highest value in the 1st quartile is around 6.8 times smaller than the standard deviation while lowest value in the fourth quartile is more than 1/2 the size of the standard deviation. 

Qualitatively this suggests that there is positive skew that most likely is impacting the shape. It therefore makes sense to take the follwoing steps:

- Check to see which companies have a significantly sized range
- Use the interquartile range to exlude outliers. 
- Graph the results 

